Primero crrer en el terminal:
mkdir -p ~/minio/data
docker run \
   -p 9000:9000 \
   -p 9001:9001 \
   --name minio1 \
   -v D:\minio\data:/data \
   -e "MINIO_ROOT_USER=ROOTUSER" \
   -e "MINIO_ROOT_PASSWORD=CHANGEME123" \
   quay.io/minio/minio server /data --console-address ":9001"
   

Para MacOs/Linux:

mkdir -p ~/minio/data
docker run \
   -p 9000:9000 \
   -p 9001:9001 \
   --name minio \
   -v ~/minio/data:/data \
   -e "MINIO_ROOT_USER=ROOTNAME" \
   -e "MINIO_ROOT_PASSWORD=CHANGEME123" \
   quay.io/minio/minio server /data --console-address ":9001"

GUI:
http://localhost:9001/login

Pasos:
1. Log in to console
2. Create a bucket
3. Create a Client User

In [1]:
#!pip install minio

# Minio buckets

In [43]:
from minio import Minio
from minio.error import S3Error

## Conexión

In [3]:
# Minio: ROOTNAME / CHANGEME123
# User "test_user": test_user / test_user

In [2]:
# Crea un cliente
client = Minio("localhost:9000",
               "ROOTNAME",
               "CHANGEME123",
               secure=False)

"""
client = Minio(endpoint="localhost:9000",
               access_key="ROOTNAME",
               secret_key="CHANGEME123",
               secure=False)
"""

'\nclient = Minio(endpoint="localhost:9000",\n               access_key="ROOTNAME",\n               secret_key="CHANGEME123",\n               secure=False)\n'

## Revisión

In [11]:
bucket_name = "test"
original_object = "img.png"
new_object = "img1.png"

### Buckets

In [66]:
# Lista de buckets
buckets = client.list_buckets()
for bucket in buckets:
    print(bucket.name, bucket.creation_date)

mybucket 2024-11-17 00:53:36.947000+00:00
test 2024-11-16 11:19:57.863000+00:00


### Objects

In [65]:
# List objects information.
objects = client.list_objects(bucket_name, recursive = True)
for obj in objects:
    print(obj)

In [67]:
# List objects information whose names starts with "my/prefix/".
objects = client.list_objects(bucket_name, prefix="my/prefix/")
for obj in objects:
    print(obj)

# List objects information recursively.
objects = client.list_objects(bucket_name, recursive=True)
for obj in objects:
    print(obj)

# List objects information recursively whose names starts with
# "my/prefix/".
objects = client.list_objects(
    bucket_name, prefix="my/prefix/", recursive=True,
)
for obj in objects:
    print(obj)

# List objects information recursively after object name
# "my/prefix/world/1".
objects = client.list_objects(
    bucket_name, recursive=True, start_after="my/prefix/world/1",
)
for obj in objects:
    print(obj)

In [68]:
# Lista objetos en el bucket
i, j = 0, 0
for bucket in buckets:
    i+=1
    print(f"{i}. Bucket '{bucket.name}':")
    bucket_name = bucket
    # Lista objetos en el bucket
    objects = client.list_objects(bucket.name)
    print(objects)
    if 'Contents' in objects:
        for obj in objects['Contents']:
            j+=1
            print(f"  {j}. Object Key: {obj['Key']}, Size: {obj['Size']} bytes, Last Modified: {obj['LastModified']}")


1. Bucket 'mybucket':
<generator object Minio._list_objects at 0x13d08a430>
2. Bucket 'test':
<generator object Minio._list_objects at 0x13d08bb30>


In [72]:
# Información de un objeto específico
bucket_name = "test"
original_object = "img.png"

result = client.stat_object(bucket_name, original_object)
print(result)

# Métodos para Procesamiento

https://github.com/minio/minio-py/tree/master/examples

In [73]:
bucket_name = "test"
original_object = "img.png"
new_object = "img1.png"

## Crear/Remover buckets

In [47]:
# Crear bucket
client.make_bucket(bucket_name)

In [48]:
# Remover bucket
client.remove_bucket(bucket_name)

## Carga/Descarga de archivos

In [76]:
# Subida de archivos
# Origen:
local_file = "file.txt"  # Ruta del archivo local
# Destino:
bucket_name = "mybucket"
object_name = "uploaded_file.txt"     # Nombre del archivo en MinIO

# Verificar si el bucket existe, y si no, crearlo
if not client.bucket_exists(bucket_name):
    client.make_bucket(bucket_name)
    print(f"Bucket '{bucket_name}' creado.")
else:
    print(f"Bucket '{bucket_name}' ya existe.")

try:
    client.fput_object(bucket_name, object_name, local_file)
    print(f"Archivo '{local_file}' subido como '{object_name}'.")
except S3Error as e:
    print(f"Error al subir archivo: {e}")

Bucket 'mybucket' ya existe.
Archivo 'file.txt' subido como 'uploaded_file.txt'.


In [32]:
# Descargar un archivo desde el bucket
# Origen:
bucket_name = "mybucket"
object_name = "uploaded_file.txt"     # Nombre del archivo en MinIO
# Destino:
downloaded_file = "downloaded_file.txt"  # Ruta para guardar el archivo descargado

try:
    client.fget_object(bucket_name, object_name, downloaded_file)
    print(f"Archivo '{object_name}' descargado como '{downloaded_file}'.")
except S3Error as e:
    print(f"Error al descargar archivo: {e}")

Archivo 'uploaded_file.txt' descargado como 'downloaded_file.txt'.


## Procesamiento de archivos

In [93]:
# Copiar objeto único
from minio.commonconfig import REPLACE, CopySource

# Origen:
source_bucket_name = "mybucket"
source_object_name = "uploaded_file.txt"     # Nombre del archivo en MinIO
# Destino:
bucket_name = "test"
object_name = "prefix/copied_file.txt"     # Nombre del archivo en MinIO

try:
    # Crea la copia en la nueva ubicación
    result = client.copy_object(bucket_name, object_name,
                            CopySource(source_bucket_name, source_object_name)
                            )
    print(f"Archivo '{source_object_name}' en '{source_bucket_name}' copiado a '{bucket_name}' como '{object_name}'.")
    
    print(f"Archivo '{source_object_name}' en '{source_bucket_name}' copiado como '{object_name}' en '{bucket_name}'.")
except S3Error as e:
    print(f"Error al copiar archivo: {e}")

Archivo 'prefix/copied_file.txt' en 'test' copiado a 'mybucket' como 'uploaded_file.txt'.
Archivo 'prefix/copied_file.txt' en 'test' copiado como 'uploaded_file.txt' en 'mybucket'.


**CUIDADO:** Por algún motivo no está funcionando para copiar hacia un bucket diferente.

In [80]:
# Borrar objeto único
bucket_name = "mybucket"
object_name = "copied_file.txt"     # Nombre del archivo en MinIO

result = client.remove_object(bucket_name, object_name)
print(f"Se elimina el archivo '{object_name}' de '{bucket_name}'.")

# Si el objeto to existe, no levanta ningún error

Se elminan el archivo 'copied_file.txt' de 'mybucket'.


In [86]:
# Borrar todos los objetos de un prefix (carpeta)
from minio.deleteobjects import DeleteObject
bucket_name = "mybucket"
prefix = "prefix/"

# Remove a prefix recursively.
delete_object_list = map(
    lambda x: DeleteObject(x.object_name),
    client.list_objects(bucket_name, prefix, recursive=True),
)
errors = client.remove_objects(bucket_name, delete_object_list)
print(f"Se eliminan todos los archivos con prefijo '{prefix}' del '{bucket_name}'.")

for error in errors:
    print("error occurred when deleting object", error)

Se eliminan todos los archivos con prefijo 'prefix/' del 'mybucket'.


In [92]:
# Mover objeto único
from minio.commonconfig import CopySource

# Origen:
source_bucket_name = "mybucket"
source_object_name = "uploaded_file.txt"     # Nombre del archivo en MinIO
# Destino:
bucket_name = "test"
object_name = "prefix/copied_file.txt"     # Nombre del archivo en MinIO

try:
    # Crea la copia en la nueva ubicación
    result = client.copy_object(bucket_name, object_name,
                            CopySource(source_bucket_name, source_object_name)
                            )
    print(f"Archivo '{source_object_name}' en '{source_bucket_name}' copiado a '{bucket_name}' como '{object_name}'.")
    
    # Borrar objeto original
    result = client.remove_object(source_bucket_name, source_object_name)
    print(f"Se elimina el archivo '{source_object_name}' de '{source_bucket_name}'.")

except S3Error as e:
    print(f"Error al copiar archivo: {e}")

Archivo 'uploaded_file.txt' en 'mybucket' copiado a 'test' como 'prefix/copied_file.txt'.
Se elimina el archivo 'uploaded_file.txt' de 'mybucket'.


## PUT/GET objects (API)

In [57]:
# Upload data
import io
bucket_name = "mybucket"
object_name = "my-object"     # Nombre del archivo en MinIO

result = client.put_object(
    bucket_name, object_name, io.BytesIO(b"hello"), 5,
)
print(
    "created {0} object; etag: {1}, version-id: {2}".format(
        result.object_name, result.etag, result.version_id,
    ),
)

created my-object object; etag: 5d41402abc4b2a76b9719d911017c592, version-id: None


In [61]:
# Get data of an object.
bucket_name = "mybucket"
object_name = "my-object"

response = None
try:
    response = client.get_object(bucket_name, object_name)
    # Read data from response.
finally:
    if response:
        response.close()
        response.release_conn()

In [59]:
response

In [64]:
# Verificación de existencia
bucket_name = "mybucket"

if client.bucket_exists(bucket_name):
    print(f"'{bucket_name}' exists")
else:
    print(f"'{bucket_name}' does not exist")

'mybucket' exists
